# 1. MODEL TRAINING

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
from pathlib import Path
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D

In [7]:
Data_dir = 'E:\project final year\pneumonia\Dataset\chest_xray'
Categories = ["Affected" , "Normal"]
for i in Categories:
    path = os.path.join(Data_dir,i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)

In [8]:
Img_size = 224
new_array = cv2.resize(img_array, (Img_size,Img_size))

In [9]:
training_data = []
def create_training_data():
    for i in Categories:
        path = os.path.join(Data_dir,i)
        class_num = Categories.index(i) ## 0 for affected 1 for Normal
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (Img_size,Img_size))
            training_data.append([new_array, class_num])
create_training_data()

In [10]:
import random
random.shuffle(training_data)

In [11]:
x = []
y = []

for features,label in training_data:
    x.append(features)
    y.append(label)
x =np.array(x).reshape(-1,Img_size,Img_size, 1)

In [12]:
pickle_out = open("x.pickle","wb")
pickle.dump(x, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [13]:
pickle_in = open("x.pickle", "rb")
x = pickle.load(pickle_in)

In [14]:
x = pickle.load(open("x.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

x = x/255.0

model = Sequential()
model.add(Conv2D(64,(3,3),input_shape=x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
x= np.array(x)
y= np.array(y)
model.fit(x, y, batch_size = 32,epochs=10,validation_split=0.3)


Epoch 1/10
129/129 [==============================] - 204s 1s/step - loss: 0.4103 - accuracy: 0.7917 - val_loss: 0.2863 - val_accuracy: 0.8902
Epoch 2/10
129/129 [==============================] - 130s 1s/step - loss: 0.1733 - accuracy: 0.9327 - val_loss: 0.2135 - val_accuracy: 0.9175
Epoch 3/10
129/129 [==============================] - 129s 1s/step - loss: 0.1211 - accuracy: 0.9554 - val_loss: 0.2049 - val_accuracy: 0.9249
Epoch 4/10
129/129 [==============================] - 129s 999ms/step - loss: 0.1022 - accuracy: 0.9636 - val_loss: 0.1556 - val_accuracy: 0.9454
Epoch 5/10
129/129 [==============================] - 129s 997ms/step - loss: 0.0700 - accuracy: 0.9758 - val_loss: 0.1436 - val_accuracy: 0.9533
Epoch 6/10
129/129 [==============================] - 129s 998ms/step - loss: 0.0538 - accuracy: 0.9805 - val_loss: 0.1816 - val_accuracy: 0.9442
Epoch 7/10
129/129 [==============================] - 129s 1s/step - loss: 0.0341 - accuracy: 0.9885 - val_loss: 0.1978 - val_accurac

In [15]:
model.save('Pneumonia_detection.h5')